In [ ]:
import yaml
import pandas as pd
import numpy as np
from pathlib import Path
import os

# ==============================================================================
# 1. CARGA DE CONFIGURAÇÃO
# ==============================================================================

def find_project_root(anchor_file="conf/config.yaml"):
    """
    Sobe os diretórios a partir do notebook atual até encontrar
    a pasta onde 'conf/config.yaml' existe.
    """
    current_path = Path.cwd()
    
    # Tenta no diretório atual e sobe até o raiz do sistema
    for parent in [current_path] + list(current_path.parents):
        potential_config = parent / anchor_file
        if potential_config.exists():
            return parent
            
    raise FileNotFoundError(f"Não foi possível encontrar a raiz do projeto contendo '{anchor_file}'.")

# 1. Definir BASE_DIR (Raiz do Projeto)
try:
    BASE_DIR = find_project_root("conf/config.yaml")
    print(f"📂 Raiz do Projeto encontrada: {BASE_DIR}")
except FileNotFoundError as e:
    # Fallback manual caso a busca automática falhe (ajuste se necessário)
    print("Busca automática falhou. Usando fallback.")
    BASE_DIR = Path("/Users/lucasborges/Downloads/TCC")

# 2. Carregar o YAML da pasta conf
CONFIG_PATH = BASE_DIR / "conf/config.yaml"
with open(CONFIG_PATH, "r") as f:
    config = yaml.safe_load(f)

# ==============================================================================
# 2. ATALHOS E VARIÁVEIS GLOBAIS
# ==============================================================================

# Atalhos dos Dicionários do YAML
# P['raw'] vai virar algo como: /Users/.../TCC/data/raw
P = {k: BASE_DIR / v for k, v in config['paths'].items()} # P de Paths
F = config['files']                                       # F de Files
PM = config['params']                                     # PM de Params

print(f"⚙️ Configuração carregada de: {CONFIG_PATH}")

# ==============================================================================
# 3. PONTE DE VARIÁVEIS
# ==============================================================================

# Caminhos de Arquivos (Apontando para o YAML)
TRAIN_EMB_PATH       = P['processed'] / F['track_embeddings']
NEW_EMB_PATH         = P['processed'] / F['new_track_embeddings']
X_TRAIN_PATH         = P['processed'] / F['train_features']
X_TEST_PATH          = P['processed'] / F['test_features']

# Ajuste conforme onde você salvou o df_tracks_complete (interim ou processed?)
# Se não estiver no YAML, usa o caminho construído:
TRACKS_COMPLETE_PATH = P['interim']   / "df_tracks_complete_v5.parquet"

# Caminhos de Grafos
# Verifica se as chaves existem no yaml, senão usa padrão
MATCHING_MAP_PATH    = P.get('graphs_coarsened', P['graphs_bipartite']) / F['matching_map']
SUPER_EMB_PATH       = P.get('graphs_super', P['graphs_bipartite'])     / F['super_embeddings']

# Parâmetros
SEED                 = PM['seed']

# Configurações Visuais Padrão
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style("whitegrid")
plt.rcParams['figure.dpi'] = 300

📂 Raiz do Projeto encontrada: /Users/lucasborges/Downloads/TCC
⚙️ Configuração carregada de: /Users/lucasborges/Downloads/TCC/conf/config.yaml


In [ ]:
# =================================================================
# 0. CARREGAMENTO E CONFIGURAÇÃO
# =================================================================
from scipy.sparse import load_npz, csr_matrix, coo_matrix
import scipy.sparse as sp
from sklearn.preprocessing import normalize
from sklearn.metrics.pairwise import cosine_similarity
import sys
from tqdm import tqdm
from numba import njit

# --- 1. Definição de Caminhos usando Config (P e F) ---
# P vem do config.yaml carregado na célula anterior
print("Mapeando arquivos...")

PATHS = {
    # Grafo e Índices 
    "B_lcc":       P['graphs_bipartite'] / "B_lcc.npz",
    "p_index":     P['graphs_bipartite'] / "p_index_lcc.parquet",
    "m_index":     P['graphs_bipartite'] / "m_index_lcc.parquet",
    
    # Features
    "feat_raw":    P['interim'] / "content_ctx_train.parquet", 
    
    # feat_scaled: caso necessário features normalizadas 
    "feat_scaled": P['processed'] / F['train_features'] # Aponta para X_train.parquet
}

# --- 2. Funções Auxiliares ---
def load_to_index(path: Path, name: str) -> pd.Index:
    """Carrega parquet e garante retorno como pd.Index limpo."""
    try:
        if not path.exists():
            raise FileNotFoundError(f"Arquivo não encontrado: {path}")
            
        data = pd.read_parquet(path)
        # Pega a primeira coluna independente do nome
        series = data.iloc[:, 0] if isinstance(data, pd.DataFrame) else data
        print(f"{name}: {len(series):,} ids carregados.")
        return pd.Index(series)
    except Exception as e:
        print(f"Erro ao carregar {name} ({path.name}): {e}")
        sys.exit(1)

def align_features(df: pd.DataFrame, target_index: pd.Index, name: str) -> pd.DataFrame:
    """Reindexa o DataFrame para bater EXATAMENTE com a ordem do grafo."""
    # Garante que track_uri é o índice
    if "track_uri" in df.columns:
        df = df.set_index("track_uri")
    
    # Verifica integridade
    missing = target_index[~target_index.isin(df.index)]
    if len(missing) > 0:
        # Se for muita coisa, erro. Se for pouco, aviso (depende da tolerância do seu TCC)
        raise ValueError(f"ERRO CRÍTICO: Faltam features em '{name}' para {len(missing)} músicas! (Ex: {missing[:3].values})")

    # Reindexação estrita (força a ordem do grafo)
    df_aligned = df.reindex(target_index)
    print(f"✓ {name} alinhado e validado: {df_aligned.shape}")
    return df_aligned

# --- 3. Execução do Carregamento ---
print("\n" + "="*70)
print("MLPb COARSENING - DATA LOADING")
print("="*70)

# A. Grafo
try:
    B_lcc = load_npz(PATHS["B_lcc"]).tocsr()
    n_playlists, n_tracks = B_lcc.shape
    print(f"Matriz B_lcc: {n_playlists:,} Playlists x {n_tracks:,} Músicas")
except FileNotFoundError:
    print(f"Erro: Grafo não encontrado em {PATHS['B_lcc']}")
    sys.exit(1)

# B. Índices
p_index_lcc = load_to_index(PATHS["p_index"], "Índice Playlists")
m_index_lcc_tracks = load_to_index(PATHS["m_index"], "Índice Músicas")

# Validação de Sanidade
assert len(p_index_lcc) == n_playlists, "Erro: Índice de playlists não bate com a matriz!"
assert len(m_index_lcc_tracks) == n_tracks, "Erro: Índice de músicas não bate com a matriz!"

# C. Features
# Raw (para cálculo de cosseno no coarsening de conteúdo)
df_raw_loaded = pd.read_parquet(PATHS["feat_raw"])
df_tracks_features_raw = align_features(df_raw_loaded, m_index_lcc_tracks, "Features RAW")

# Scaled (caso precise para conferência ou método alternativo)
df_scaled_loaded = pd.read_parquet(PATHS["feat_scaled"])
df_tracks_features_scaled = align_features(df_scaled_loaded, m_index_lcc_tracks, "Features SCALED")

# D. Grau 
track_degrees = np.asarray(B_lcc.sum(axis=0)).ravel()
print(f"Grau Médio Músicas: {track_degrees.mean():.2f}")
print(f"Máx Grau: {track_degrees.max()} | Mín Grau: {track_degrees.min()}")

Mapeando arquivos...

MLPb COARSENING - DATA LOADING
✓ Matriz B_lcc: 98,726 Playlists x 324,132 Músicas
✓ Índice Playlists: 98,726 ids carregados.
✓ Índice Músicas: 324,132 ids carregados.
✓ Features RAW alinhado e validado: (324132, 49)
✓ Features SCALED alinhado e validado: (324132, 49)
✓ Grau Médio Músicas: 1.14
✓ Máx Grau: 412.19632894993583 | Mín Grau: 0.06419019640788683


In [39]:
# =================================================================
# 1. PESO DO NÓ (SIGMA)
# =================================================================

def prepare_node_weights_vectorized(df_features_raw: pd.DataFrame, track_index: pd.Index):
    print("\n=== Calculando pesos (sigma) ===")
    if "artist_followers" not in df_features_raw.columns:
         raise ValueError("DataFrame deve conter coluna 'artist_followers'")

    series_raw = df_features_raw["artist_followers"]
    series_aligned = series_raw.reindex(track_index)
    
    # Imputação
    if series_aligned.isna().sum() > 0:
        series_aligned = series_aligned.fillna(series_aligned.mean())

    # Log-Scale Normalizado
    denom = np.log1p(series_aligned.mean()) 
    weights_array = np.log1p(series_aligned.values) / denom
    weights_array = weights_array.astype(np.float64)

    print(f"Peso Total: {weights_array.sum():.2f} | Médio: {weights_array.mean():.4f}")
    return weights_array

# Execução
sigma_weights_array = prepare_node_weights_vectorized(df_tracks_features_raw, m_index_lcc_tracks)


=== Calculando pesos (sigma) ===
Peso Total: 258998.33 | Médio: 0.7991


In [41]:
# =================================================================
# 2. PARÂMETROS
# =================================================================

ZETA_TARGET = 20_000     # Alvo dos super-nós
UPPER_BOUND = 0.5        # Margem de tolerância de peso
T_MAX = 15               # Máx iterações
TAU_FRAC = 0.05          # Tolerância de convergência
DEGREE_THRESHOLD = 2     # Divisão Core vs Tail
SEED_PRIORITY = "degree" 

print(f"Configuração: Zeta={ZETA_TARGET:,}, UpperBound={UPPER_BOUND}, DegreeThresh={DEGREE_THRESHOLD}")

Configuração: Zeta=20,000, UpperBound=0.5, DegreeThresh=2


In [42]:
# =================================================================
# 3. VIZINHOS 2-HOP (VETORIZADO)
# =================================================================

def compute_neighbors_vectorized(
    B_sparse: sp.csr_matrix,
    track_indices=None, # None = Todos
    top_k: int = 100,
    min_weight: float = 0.0,
    batch_size: int = 1000,
    playlist_sample_size: int = 500
) -> sp.csr_matrix:
    print(f"\n=== Pré-cálculo de Vizinhos ===")
    n_playlists, n_tracks = B_sparse.shape
    
    # Amostragem de Hubs
    B_processed = B_sparse.tocsc()
    if playlist_sample_size > 0:
        rng = np.random.default_rng(42)
        degrees = np.diff(B_processed.indptr)
        hubs = np.where(degrees > playlist_sample_size)[0]
        if len(hubs) > 0:
            print(f"📉 Amostrando {len(hubs)} hubs...")
            new_data, new_indices, new_indptr = [], [], [0]
            for t in tqdm(range(n_tracks), desc="Sampling"):
                start, end = B_processed.indptr[t], B_processed.indptr[t+1]
                indices = B_processed.indices[start:end]
                if len(indices) > playlist_sample_size:
                    chosen = rng.choice(indices, size=playlist_sample_size, replace=False)
                    chosen.sort()
                    new_indices.extend(chosen)
                    new_data.extend([1]*len(chosen))
                    new_indptr.append(new_indptr[-1] + len(chosen))
                else:
                    new_indices.extend(indices)
                    new_data.extend(B_processed.data[start:end])
                    new_indptr.append(new_indptr[-1] + (end-start))
            B_processed = sp.csc_matrix((new_data, new_indices, new_indptr), shape=B_processed.shape)

    # Similaridade (Cosseno via Dot Product)
    Bt = B_processed.transpose().tocsr()
    Bt_norm = normalize(Bt, norm='l2', axis=1)
    
    target_indices = np.arange(n_tracks) if track_indices is None else track_indices
    if not np.issubdtype(target_indices.dtype, np.integer):
         target_indices = np.arange(n_tracks) # Fallback se vier strings

    final_rows, final_cols, final_data = [], [], []

    print(f"Calculando Similaridade...")
    for i in tqdm(range(0, len(target_indices), batch_size)):
        batch_idx = target_indices[i : i + batch_size]
        sim_block = Bt_norm[batch_idx, :].dot(Bt_norm.T)
        
        # Zera diagonal
        for local_r, global_c in enumerate(batch_idx):
            sim_block[local_r, global_c] = 0.0
            
        if min_weight > 0:
            sim_block.data[sim_block.data < min_weight] = 0
            sim_block.eliminate_zeros()

        # Top-K manual (rápido p/ esparsas)
        for r in range(sim_block.shape[0]):
            start, end = sim_block.indptr[r], sim_block.indptr[r+1]
            if start == end: continue
            
            idx = sim_block.indices[start:end]
            dat = sim_block.data[start:end]
            
            if len(dat) > top_k:
                top_args = np.argpartition(dat, -top_k)[-top_k:]
                idx, dat = idx[top_args], dat[top_args]
            
            final_rows.extend([batch_idx[r]] * len(idx))
            final_cols.extend(idx)
            final_data.extend(dat)

    S_csr = sp.csr_matrix((final_data, (final_rows, final_cols)), shape=(n_tracks, n_tracks))
    print(f"✓ Matriz Vizinhos: {S_csr.nnz:,} arestas")
    return S_csr

# Execução (sem argumento track_indices para evitar erro)
S_wcn_matrix = compute_neighbors_vectorized(B_lcc, top_k=100, playlist_sample_size=500)


=== Pré-cálculo de Vizinhos ===
📉 Amostrando 886 hubs...


Sampling: 100%|██████████| 324132/324132 [00:00<00:00, 611157.18it/s]


Calculando Similaridade...


100%|██████████| 325/325 [00:12<00:00, 25.10it/s]


✓ Matriz Vizinhos: 23,576,404 arestas


In [43]:
# =================================================================
# 4. MLPb
# =================================================================

@njit
def _run_label_propagation_fast(
    indptr, indices, data, sigma_array, initial_labels, initial_cluster_weights,
    node_order, S_max, min_vertices, max_iter, tolerance_abs
):
    labels = initial_labels.copy()
    cluster_weights = initial_cluster_weights.copy()
    num_nodes = len(labels)
    num_sv_active = np.sum(cluster_weights > 0)

    for itr in range(max_iter):
        swap_count = 0
        for i in range(num_nodes):
            u = node_order[i]
            old_label = labels[u]
            u_weight = sigma_array[u]
            
            start, end = indptr[u], indptr[u+1]
            if start == end: continue
            
            neighbors = indices[start:end]
            scores = data[start:end]
            
            # Map manual para Numba
            unique_lbls = np.empty(len(neighbors), dtype=np.int32)
            unique_scores = np.empty(len(neighbors), dtype=np.float32)
            count = 0
            
            for k in range(len(neighbors)):
                lbl_v = labels[neighbors[k]]
                w_sim = scores[k]
                
                # Check Capacidade
                w_check = cluster_weights[lbl_v]
                if lbl_v != old_label: w_check += u_weight
                
                if w_check <= S_max:
                    found = False
                    for existing in range(count):
                        if unique_lbls[existing] == lbl_v:
                            unique_scores[existing] += w_sim
                            found = True; break
                    if not found:
                        unique_lbls[count] = lbl_v
                        unique_scores[count] = w_sim
                        count += 1
            
            # Argmax
            max_val = -1.0
            target = old_label
            # Score do label atual para comparação
            for k in range(count):
                if unique_lbls[k] == old_label:
                    max_val = unique_scores[k]; break
            
            for k in range(count):
                if unique_scores[k] > max_val:
                    max_val = unique_scores[k]
                    target = unique_lbls[k]
            
            if target != old_label:
                cluster_weights[old_label] -= u_weight
                cluster_weights[target] += u_weight
                labels[u] = target
                swap_count += 1
                if cluster_weights[old_label] <= 0: num_sv_active -= 1
                if num_sv_active <= min_vertices: return labels, cluster_weights
        
        if swap_count <= tolerance_abs: break
            
    return labels, cluster_weights

def mlpb_coarsening_vectorized(
    S_wcn_matrix, sigma_array, zeta_target=None, upper_bound=0.2, 
    max_iter=15, tolerance_frac=0.05, seed_priority="degree"
):
    print(f"\n=== MLPb Coarsening ===")
    n_nodes = S_wcn_matrix.shape[0]
    min_vertices = min(zeta_target, n_nodes) if zeta_target else max(1, n_nodes//2)
    
    # S_max baseado na mediana
    sigma_median = np.median(sigma_array)
    target_size = n_nodes / float(min_vertices)
    S_max = (1.0 + upper_bound) * target_size * sigma_median
    
    print(f"Zeta: {min_vertices:,} | S_max: {S_max:.4f}")
    
    # Setup
    initial_labels = np.arange(n_nodes, dtype=np.int32)
    initial_weights = sigma_array.copy().astype(np.float64)
    
    indices = np.arange(n_nodes)
    if seed_priority == "degree":
        degs = np.array(S_wcn_matrix.sum(axis=1)).ravel()
        node_order = np.argsort(-degs).astype(np.int32)
    else:
        node_order = np.random.permutation(n_nodes).astype(np.int32)
        
    print("Rodando LP...")
    lbls, w = _run_label_propagation_fast(
        S_wcn_matrix.indptr, S_wcn_matrix.indices, S_wcn_matrix.data.astype(np.float32),
        initial_weights, initial_labels, initial_weights, node_order,
        float(S_max), int(min_vertices), max_iter, int(tolerance_frac * n_nodes)
    )
    return lbls, w

In [29]:
# =================================================================
# 5. CONTENT-BASED MATCHING (BEST EFFORT)
# =================================================================

def content_based_matching_vectorized(
    low_degree_indices, df_features_aligned, matching_high_degree, 
    cluster_weights, sigma_array, S_max
):
    print(f"\n=== Content Matching (Best Effort) ===")
    
    # 1. Features
    cols = [c for c in df_features_aligned.select_dtypes(include=[np.number]).columns 
            if c not in {'track_uri', 'artist_uri', 'p_index', 'm_index'}]
    
    if not cols: return {idx: f"L_new_{i}" for i, idx in enumerate(low_degree_indices)}
    X_all = df_features_aligned[cols].values
    
    # 2. Centróides
    print("  Calculando centróides...")
    high_idxs = list(matching_high_degree.keys())
    high_lbls = list(matching_high_degree.values())
    
    df_centroids = pd.DataFrame(X_all[high_idxs], columns=cols)
    df_centroids['label'] = high_lbls
    df_centroids = df_centroids.groupby('label').mean()
    
    centroids_mtx = df_centroids.values
    labels_list = df_centroids.index.tolist()
    curr_weights = np.array([cluster_weights.get(l, 0.0) for l in labels_list])
    
    # 3. Matching
    matching_low = {}
    forced = 0
    new_counter = 0
    BATCH = 5000
    
    print("  Atribuindo...")
    for start in tqdm(range(0, len(low_degree_indices), BATCH)):
        end = min(start + BATCH, len(low_degree_indices))
        batch_idxs = low_degree_indices[start:end]
        
        batch_feat = X_all[batch_idxs]
        batch_w = sigma_array[batch_idxs]
        
        sims = cosine_similarity(batch_feat, centroids_mtx)
        pot_w = batch_w[:, None] + curr_weights[None, :]
        valid_mask = pot_w <= S_max
        
        for i in range(len(batch_idxs)):
            node_idx = batch_idxs[i]
            node_w = batch_w[i]
            
            # Tenta válido
            valid_sims = np.where(valid_mask[i], sims[i], -np.inf)
            best_idx = np.argmax(valid_sims)
            
            if valid_sims[best_idx] == -np.inf:
                # FALLBACK: Força no mais similar
                best_idx = np.argmax(sims[i])
                forced += 1
            
            lbl = labels_list[best_idx]
            matching_low[node_idx] = lbl
            curr_weights[best_idx] += node_w # Atualiza peso
            
    print(f"✓ Matching Final. Forçados: {forced:,}")
    return matching_low

In [44]:
# =================================================================
# 6. PIPELINE PRINCIPAL
# =================================================================

print("="*70)
print("ORCHESTRATION")
print("="*70)

# 1. Split
high_degree_mask = track_degrees >= DEGREE_THRESHOLD
global_high_idx = np.where(high_degree_mask)[0]
global_low_idx = np.where(~high_degree_mask)[0]

print(f"Core: {len(global_high_idx):,} | Tail: {len(global_low_idx):,}")

# 2. MLPb no Core
S_core = S_wcn_matrix[global_high_idx, :][:, global_high_idx]
sigma_core = sigma_weights_array[global_high_idx]

local_lbls, local_w = mlpb_coarsening_vectorized(
    S_core, sigma_core, ZETA_TARGET, UPPER_BOUND, T_MAX, TAU_FRAC, SEED_PRIORITY
)

# Mapeamento Core
matching_high = {}
weights_dict = {}
for i, l in enumerate(local_lbls):
    gid = global_high_idx[i]
    lbl_str = f"SV_{l}"
    matching_high[gid] = lbl_str
    weights_dict[lbl_str] = float(local_w[l])

# S_max recalculado para consistência
_target_sz = len(global_high_idx) / float(min(ZETA_TARGET, len(global_high_idx)))
S_MAX_ACTUAL = (1.0 + UPPER_BOUND) * _target_sz * np.median(sigma_core)

# 3. Matching na Tail
matching_low = content_based_matching_vectorized(
    global_low_idx, df_tracks_features_scaled, matching_high, 
    weights_dict, sigma_weights_array, S_MAX_ACTUAL
)

# 4. Merge
final_matching = {**matching_high, **matching_low}
print(f"Total Músicas: {len(final_matching):,} -> Super-Nós: {len(set(final_matching.values())):,}")

ORCHESTRATION
Core: 21,905 | ❄️ Tail: 302,227

=== MLPb Coarsening ===
Zeta: 20,000 | S_max: 1.6314
Rodando LP...

=== Content Matching (Best Effort) ===
  Calculando centróides...
  Atribuindo...


100%|██████████| 61/61 [00:38<00:00,  1.59it/s]

✓ Matching Final. Forçados: 252,404
Total Músicas: 324,132 -> Super-Nós: 20,047


In [45]:
# =================================================================
# 7. CONTRAÇÃO (MATRICIAL)
# =================================================================

def contract_graph_vectorized(B_sparse, matching_dict):
    print(f"\n=== Contração ===")
    n_pl, n_tr = B_sparse.shape
    
    unique_lbls = sorted(list(set(matching_dict.values())))
    lbl_map = {l: i for i, l in enumerate(unique_lbls)}
    n_super = len(unique_lbls)
    
    # Matriz de Projeção P (Tracks -> Super)
    rows, cols = [], []
    for t, l in matching_dict.items():
        if t < n_tr:
            rows.append(t)
            cols.append(lbl_map[l])
            
    P = sp.csr_matrix((np.ones(len(rows)), (rows, cols)), shape=(n_tr, n_super))
    
    # Contração: B_red = B @ P
    B_red = B_sparse.dot(P)
    
    print(f"Original: {B_sparse.shape} -> Reduzido: {B_red.shape}")
    print(f"Compressão Arestas: {B_sparse.nnz/B_red.nnz:.1f}x")
    
    return B_red, pd.Index(unique_lbls, name="super_track_id")

B_coarsened, super_m_index_new = contract_graph_vectorized(B_lcc, final_matching)


=== Contração ===
Original: (98726, 324132) -> Reduzido: (98726, 20047)
Compressão Arestas: 1.0x


In [46]:
# =================================================================
# 8. MÉTRICAS DE QUALIDADE DO COARSENING
# =================================================================

import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix

def compute_quality_metrics_vectorized(
    B_original: csr_matrix, 
    B_coarsened: csr_matrix, 
    matching_dict: dict, 
    sigma_array: np.ndarray,   # Array numpy (pesos originais)
    track_degrees: np.ndarray  # Array numpy (graus originais)
):
    print(f"\n=== Métricas de Qualidade ===")

    # 1. Preparação dos Dados (DataFrame para agregação rápida)
    # ---------------------------------------------------------
    tracks = list(matching_dict.keys())
    labels = list(matching_dict.values())
    
    df_metrics = pd.DataFrame({
        'original_track_idx': tracks,
        'super_label': labels
    })
    
    # Mapeia propriedades originais
    df_metrics['weight'] = sigma_array[df_metrics['original_track_idx']]
    df_metrics['degree'] = track_degrees[df_metrics['original_track_idx']]
    
    # 2. Agregação por Super-Nó (GroupBy)
    # -----------------------------------
    # Calcula: Tamanho (count), Peso Total (sum), Grau Interno Médio (mean)
    super_stats = df_metrics.groupby('super_label').agg(
        size=('original_track_idx', 'count'),
        total_weight=('weight', 'sum'),
        avg_internal_degree=('degree', 'mean')
    )
    
    # Arrays para cálculos estatísticos
    sizes = super_stats['size'].values
    weights = super_stats['total_weight'].values
    internal_degrees = super_stats['avg_internal_degree'].values
    
    # Grau do grafo REDUZIDO (quantas playlists cada super-nó conecta)
    super_degrees = np.asarray(B_coarsened.sum(axis=0)).ravel()

    # 3. Cálculos Gerais
    # ------------------
    orig_tracks = B_original.shape[1]
    coar_tracks = B_coarsened.shape[1]
    red_tracks = orig_tracks / coar_tracks if coar_tracks > 0 else 0

    orig_edges = B_original.nnz
    coar_edges = B_coarsened.nnz
    red_edges = orig_edges / coar_edges if coar_edges > 0 else 0

    density_orig = orig_edges / (B_original.shape[0] * orig_tracks)
    density_coar = coar_edges / (B_coarsened.shape[0] * coar_tracks)
    
    # Tamanho teórico da matriz (Linhas * Colunas)
    orig_size = B_original.shape[0] * orig_tracks
    coar_size = B_coarsened.shape[0] * coar_tracks
    compression_ratio = orig_size / coar_size if coar_size > 0 else 0
    
    effective_reduction = red_tracks / np.mean(sizes)

    # 4. Impressão dos Resultados (Formato original)
    # ----------------------------------------------
    print(f"\n--- Taxas de Redução ---")
    print(f"Músicas: {orig_tracks:,} → {coar_tracks:,} ({red_tracks:.1f}x)")
    print(f"Arestas: {orig_edges:,} → {coar_edges:,} ({red_edges:.1f}x)")

    print(f"\n--- Distribuição de Super-Nós ---")
    print(f"Tamanho médio: {np.mean(sizes):.1f} músicas")
    print(f"Tamanho [min, med, max]: [{np.min(sizes)}, {int(np.median(sizes))}, {np.max(sizes)}]")
    print(f"Grau médio interno: {np.mean(internal_degrees):.1f}")
    print(f"Peso médio (Sigma): {np.mean(weights):.2f}")
    print(f"Peso [min, med, max]: [{np.min(weights):.2f}, {np.median(weights):.2f}, {np.max(weights):.2f}]")
    
    # Coeficiente de Variação (para checar balanceamento)
    cv = (np.std(weights) / np.mean(weights)) if np.mean(weights) > 0 else 0
    print(f"Coef. Variação Peso (CV): {cv:.2f} (Ideal: baixo)")

    print(f"\n--- Densidade ---")
    print(f"Original: {density_orig:.6e}")
    print(f"Coarsened: {density_coar:.6e}")
    print(f"Razão: {density_coar/density_orig:.3f}x")

    print(f"\n--- Grau das Super-Músicas (Grafo Reduzido) ---")
    print(f"Média: {super_degrees.mean():.1f}")
    print(f"Mediana: {np.median(super_degrees):.0f}")
    print(f"[Min, Max]: [{super_degrees.min()}, {super_degrees.max()}]")

    print(f"\n--- Compressão ---")
    print(f"Tamanho da matriz original: {orig_size:,} células")
    print(f"Tamanho da matriz coarsened: {coar_size:,} células")
    print(f"Taxa de compressão: {compression_ratio:.1f}x")

    print(f"\n--- Eficiência ---")
    print(f"Redução efetiva por super-nó: {effective_reduction:.2f}x")

    # Retorna dicionário completo
    return {
        "reduction_tracks": red_tracks,
        "reduction_edges": red_edges,
        "super_node_count": len(sizes),
        "avg_super_node_size": np.mean(sizes),
        "median_super_node_size": np.median(sizes),
        "max_super_node_size": np.max(sizes),
        "avg_super_node_weight": np.mean(weights),
        "std_super_node_weight": np.std(weights),
        "cv_weight": cv,
        "density_ratio": density_coar/density_orig,
        "compression_ratio": compression_ratio,
        "effective_reduction": effective_reduction,
        # Retorna o DF caso queira salvar detalhes depois
        "stats_df": super_stats
    }

# Execução
metrics = compute_quality_metrics_vectorized(
    B_original=B_lcc,
    B_coarsened=B_coarsened,
    matching_dict=final_matching,
    sigma_array=sigma_weights_array,
    track_degrees=track_degrees
)


=== Métricas de Qualidade ===

--- Taxas de Redução ---
Músicas: 324,132 → 20,047 (16.2x)
Arestas: 2,969,357 → 2,861,520 (1.0x)

--- Distribuição de Super-Nós ---
Tamanho médio: 16.2 músicas
Tamanho [min, med, max]: [1, 9, 845]
Grau médio interno: 3.1
Peso médio (Sigma): 12.92
Peso [min, med, max]: [1.28, 7.78, 852.16]
Coef. Variação Peso (CV): 1.47 (Ideal: baixo)

--- Densidade ---
Original: 9.279166e-05
Coarsened: 1.445825e-03
Razão: 15.581x

--- Grau das Super-Músicas (Grafo Reduzido) ---
Média: 18.5
Mediana: 10
[Min, Max]: [2.0982708135852413, 717.6245368767895]

--- Compressão ---
Tamanho da matriz original: 32,000,255,832 células
Tamanho da matriz coarsened: 1,979,160,122 células
Taxa de compressão: 16.2x

--- Eficiência ---
Redução efetiva por super-nó: 1.00x


In [52]:
# =================================================================
# 9. SALVAMENTO
# =================================================================

import scipy.sparse as sp

def save_coarsening_results_optimized(
    output_dir: Path, # <--- MUDANÇA 1: Recebe a pasta final, não a base
    B_coarsened: sp.csr_matrix,
    super_m_index: pd.Index,
    final_matching: dict,
    m_index_original: pd.Index,
    metrics_data: dict,
    original_nnz: int,
    original_shape: tuple
):
    """
    Salva os resultados do Coarsening na pasta indicada pelo config.
    """
    print(f"\n=== Salvando Resultados (Vetorizado) ===")
    
    # 1. Garantir que o diretório existe
    output_dir.mkdir(parents=True, exist_ok=True)
    print(f"📁 Diretório de Saída: {output_dir}")

    # 2. Salvar Matriz Esparsa (NPZ)
    sp.save_npz(output_dir / "B_coarsened.npz", B_coarsened)
    print(f"✓ Matriz B_coarsened salva: {B_coarsened.shape}")

    # 3. Salvar Índice das Super-Músicas
    super_m_index.name = "super_track_id"
    super_m_index.to_frame().to_parquet(output_dir / "super_m_index.parquet")
    print(f"✓ Índice super_m_index salvo.")

    # 4. Salvar Mapa de Matching
    print("  Construindo DataFrame de matching...")
    
    # final_matching é {int_idx: 'label'}
    df_matching = pd.DataFrame.from_dict(
        final_matching, 
        orient='index', 
        columns=['super_track_id']
    )
    df_matching.index.name = 'original_track_idx'
    
    # Recuperar URIs originais usando acesso posicional rápido
    if hasattr(m_index_original, 'values'):
        uris_array = m_index_original.values
    else:
        uris_array = np.array(m_index_original)
        
    # Indexação segura
    df_matching['original_track_uri'] = uris_array[df_matching.index]
    
    # Reordenar para ficar bonito
    df_matching = df_matching[['original_track_uri', 'super_track_id']]
    
    # Usa o nome do arquivo definido no YAML (F['matching_map']) se disponível, senão default
    filename_map = F.get('matching_map', "matching_map.parquet")
    df_matching.to_parquet(output_dir / filename_map)
    print(f"✓ Matching Map salvo em '{filename_map}': {len(df_matching):,} registros.")

    # 5. Salvar Métricas e Stats
    metrics_clean = {k: v for k, v in metrics_data.items() if not isinstance(v, pd.DataFrame)}
    
    stats_full = {
        **metrics_clean,
        "original_tracks": original_shape[1],
        "original_edges": original_nnz,
        "coarsened_tracks": B_coarsened.shape[1],
        "coarsened_edges": B_coarsened.nnz,
        "compression_ratio": original_shape[1] / B_coarsened.shape[1] if B_coarsened.shape[1] > 0 else 0,
        "timestamp": pd.Timestamp.now().isoformat()
    }
    
    pd.DataFrame([stats_full]).to_parquet(output_dir / "coarsening_stats.parquet")
    print(f"Estatísticas salvas.")
    
    print("\n" + "="*70)
    print("PIPELINE DE COARSENING CONCLUÍDO COM SUCESSO")
    print("="*70)

# =================================================================
# USO FINAL 
# =================================================================


save_coarsening_results_optimized(
    output_dir=P['graphs_coarsened'], 
    B_coarsened=B_coarsened,
    super_m_index=super_m_index_new,
    final_matching=final_matching,
    m_index_original=m_index_lcc_tracks,
    metrics_data=metrics, 
    original_nnz=B_lcc.nnz,
    original_shape=B_lcc.shape
)


=== Salvando Resultados (Vetorizado) ===
📁 Diretório de Saída: /Users/lucasborges/Downloads/TCC/graphs/bipartite/coarsened
✓ Matriz B_coarsened salva: (98726, 20047)
✓ Índice super_m_index salvo.
  Construindo DataFrame de matching...
✓ Matching Map salvo em 'matching_map.parquet': 324,132 registros.
Estatísticas salvas.

PIPELINE DE COARSENING CONCLUÍDO COM SUCESSO
